<a href="https://colab.research.google.com/github/msjun23/Deep-Learning-from-Scratch/blob/main/Chapter3/output_layer_design.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#출력층 설계하기
신경망은 분류와 회귀 모두에 이용할 수 있다.

- 분류 : 데이터가 어느 클래스에 속하는냐 -> **소프트맥스**
- 회귀 : 입력 데이터에서 연속적인 수치 예측 -> **항등 함수**

#항등 함수

항등 함수(identity function)는 입력을 그대로 출력한다. '항등'이라는 뜻 그대로 입력과 출력이 항상 같다.

#소프트맥스 함수
소프트맥스(softmax) 함수는 아래와 같다.

>$y_k=\frac{exp(a_k)}{\sum_{i=1}^{n}exp(a_i)}$

$n$은 추력층의 뉴런 수, $y_k$는 그중 $k$번째 출력임을 뜻한다. 분자는 입력 신호 $a_k$의 지수 함수, 분모는 모든 입력 신호의 지수 함수의 합으로 구성된다.

위 식의 분모를 보면 알 수 있듯이, 소프트맥스의 출력은 모든 입력 신호로부터 영향을 받는다.

In [1]:
import numpy as np

In [2]:
# 소프트맥스 구현하기
a = np.array([0.3, 2.9, 4.0])
exp_a = np.exp(a)   # 각 입력에 대한 지수 함수
print(exp_a)

sum_exp_a = np.sum(exp_a)
print(sum_exp_a)

y = exp_a / sum_exp_a
print(y)

[ 1.34985881 18.17414537 54.59815003]
74.1221542101633
[0.01821127 0.24519181 0.73659691]


In [3]:
# 소프트맥스 함수 정의
def softmax(a):
  exp_a = np.exp(a)
  sum_exp_a = np.sum(exp_a)
  y = exp_a / sum_exp_a

  return y

위와 같이 소프트맥스 함수를 구현하고 정의했다. 하지만 한 가지 주의해야 할 점이 있다. 소프트맥스 함수는 **지수 함수**를 사용하는데 지수 함수는 입력이 조금만 증가해도 아주 큰 값을 출력한다. 예를 들어 $e^{10}$은 20,000이 넘고, $e^{1000}$은 무한대를 뜻하는 inf를 출력한다. 컴퓨터가 표현할 수 있는 범위가 문제가 되어 쉽게 **오버플로우**가 발생할 수 있다.

이 문제를 해결하기 위해 소프트맥스의 식을 조금 개선해야 한다.

> $$y_k=\frac{exp(a_k)}{\sum_{i=1}^{n}exp(a_i)}=\frac{Cexp(a_k)}{C\sum_{i=1}^{n}exp(a_i)}
\\=\frac{exp(a_k+logC)}{\sum_{i=1}^{n}exp(a_i+logC)}
\\=\frac{exp(a_k+C')}{\sum_{i=1}^{n}exp(a_i+C')}$$

분자, 분모에 $C$라는 임의의 정수를 곱해주었다. 그 후는 단순히 수학적인 전개이다.

위의 식에서 알 수 있는 것은 **소프트맥스의 지수 함수를 계산할 때 어떤 정수를 더해도(빼도) 결과는 바뀌지 않는다**는 것이다.

In [4]:
# 소프트맥스 오버플로 예시
a = np.array([1010, 1000, 990])
print(np.exp(a) / np.sum(np.exp(a)))

c = np.max(a)
print(a - c)

print(np.exp(a - c) / np.sum(np.exp(a - c)))

[nan nan nan]
[  0 -10 -20]
[9.99954600e-01 4.53978686e-05 2.06106005e-09]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


예시에서 확인할 수 있듯이, 큰 수에 대해 별다른 조치없이 계산하면 nan(not a number)이 출력된다. 하지만 이를 방지하기 위해 입력 신호 중 최대값(c)를 빼주면 올바른 계산이 가능했다.

In [5]:
# 소프트맥스 재정의
def softmax(a):
  c = np.max(a)
  exp_a = np.exp(a - c)   # 오버플로 대책(결과는 바뀌지 않는다.)
  sum_exp_a = np.sum(exp_a)
  y = exp_a / sum_exp_a
  
  return y

In [6]:
# 소프트맥스 출력 예시
a = np.array([0.3, 2.9, 4.0])
y = softmax(a)
print(y)

print(np.sum(y))

[0.01821127 0.24519181 0.73659691]
1.0


위 예시를 보면 소프트맥스 함수의 **모든 출력은 0 에서 1 사이**이다. 또한 **출력의 총합은 1** 이다. 이는 소프트맥스에 있어 상당히 중요한 성질이다. 이 성질 덕분에 소프트맥스 함수의 출력을 **'확률'**로 해석할 수 있다.

예시를 보면 y[0]의 확률은 0.018(1.8%), y[1], y[2]는 각각 0.245(24.5%), 0.737(73.7%)로 해석 가능하다. 이 결과는 입력의 답은 73.7%의 확률로 2번째 클래스라는 것을 암시한다. 즉, 소프트맥스 함수를 이용함으로써 문제를 **확률적**(통계적)으로 대응할 수 있게됐다.

여기서 한가지 주의할 점이 있다.소프트맥스 함수를 적용해도 **각 원소의 대소 관계는 변하지 않는다**. $y=exp(x)$는 단수 증가 함수이기 때문이다. 실제로 앞의 예시에서 $a$의 원소들 중 가장 큰 값은 2번째 원소이고, $y$의 가장 큰 원소 또한 2번째 원소이다.

신경망을 이용한 분류에서는 일반적으로 가장 큰 출력을 내는 뉴런에 해당하는 클래스만을 인식한다. 그리고 위의 특징에 따라 소프트맥스 함수를 적용해도 출력이 가장 큰 뉴런의 위치는 달라지지 않는다. 결과적으로 **신경망으로 분류**할 때는 출력층의 **소프트맥스 함수를 생략**해도 된다는 것이다. 현업에서도 지수 함수 계산에 들어가는 자원 낭비를 줄이고자 출력층의 소프트맥스 함수를 생략하는 것이 일반적이다.
> 신경망을 학습시킬 때는 출력층에서 소프트맥스 함수를 사용한다. 출력층의 소프트맥스 함수를 생략하는 것이 일반적인것은 추론 단계이다.

#출력층의 뉴런 수

출력층의 뉴런 수는 풀려는 문제에 맞게 정해야한다. 분류에서는 분류하고자 하는 클래스의 수로 설정하는 것이 일반적이다. 예를 들어 입력이미지를 0 ~ 9 중 하나의 숫자로 분류하고자 한다면 출력층의 뉴런 수를 10개로 설정한다.